In [1]:
#0. Preliminaries
import pandas as pd
import nltk
nltk.download('punkt')
from readability import Readability

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [6]:
#1. Import Data
#chunked test from data_generation, needs to be redone 
df = pd.read_csv("../data/author_data.csv")

In [7]:
df

,Author,Title,Year,Link,Abstract,Introduction,Conclusion
0,Aman Madaan,AutoMix: Automatically Mixing Language Models,2023,https://arxiv.org/pdf/2310.12963.pdf,Large language models (LLMs) are now available...,NaN,NaN
1,Aman Madaan,Self-refine: Iterative refinement with self-fe...,2023,https://arxiv.org/pdf/2303.17651,"Like humans, large language models (LLMs) do n...",Although large language models (LLMs) can gene...,We present SELF-REFINE: a novel approach that ...
2,Aman Madaan,Learning performance-improving code edits,2023,https://arxiv.org/pdf/2302.07867,The waning of Mooreâs Law has shifted the fo...,NaN,NaN
3,Aman Madaan,Language models of code are few-shot commonsen...,2022,https://arxiv.org/pdf/2210.07128,We address the general task of structured comm...,The growing capabilities of large pre-trained ...,We present the first work to employ large lang...
4,Aman Madaan,Text and patterns: For effective chain of thou...,2022,https://arxiv.org/pdf/2209.07686,"In the past decade, we witnessed dramatic gain...",The ability to learn a previously unseen task ...,This work evaluates the capacity of COT to ele...
...,...,...,...,...,...,...,...
59,Timo Schick,BERTRAM: Improved word embeddings have big imp...,2019,https://arxiv.org/pdf/1910.07181.pdf,Pretraining deep language models has led to la...,As word embedding algorithms (e.g. Mikolov et ...,"We have introduced BERTRAM, a novel architectu..."
60,Timo Schick,True few-shot learning with promptsâa real-w...,2022,NaN,Prompt-based approaches are strong at fewshot ...,With pretrained language models (LMs) getting\...,In light of recent work casting doubt on the p...
61,Timo Schick,Learning semantic representations for novel wo...,2019,https://arxiv.org/pdf/1811.03866.pdf,Word embeddings are a key component of high-pe...,Distributed word representations (or embedding...,We have presented a model that is capable of i...
62,Timo Schick,Transition-based generation from abstract mean...,2017,https://arxiv.org/pdf/1707.07591.pdf,This work addresses the task of generating Eng...,Semantic representations of natural language a...,We have devised a novel approach for the chall...


In [ ]:
#Flesch 
def flesch_readability_scale(text):
    r = Readability(text)
    f = r.flesch()
    return f.score
    
df['flesch_score'] = df['text'].apply(flesch_readability_scale)